In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import time

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv('../input/finaltrain/final_train.csv')
train.head()

In [ ]:
test = pd.read_csv('../input/final-test/final_test.csv')
test.head()

In [ ]:
del train['Unnamed: 0']
del test['Unnamed: 0']

In [ ]:
#train_s = train.sample(n=1000000)

train_s.apply(lambda x: sum(x.isnull()),axis=0)

train_s.is_churn.value_counts()

(76764/923236)*100

test.apply(lambda x: sum(x.isnull()),axis=0)

train_s.columns

In [ ]:
###### #########Porto-Logic ####

In [ ]:
#del test['Unnamed: 0']
train.head()

In [ ]:
test_id = test['msno'] ## for backup 
is_churn = train['is_churn']  ## for backup ##

In [ ]:
# This function late in a list of features 'cols' from train and test dataset, 
# and performing frequency encoding. 
def freq_encoding(cols, train_df, test_df):
    # we are going to store our new dataset in these two resulting datasets
    result_train_df=pd.DataFrame()
    result_test_df=pd.DataFrame()
    
    # loop through each feature column to do this
    for col in cols:
        
        # capture the frequency of a feature in the training set in the form of a dataframe
        col_freq=col+'_freq'
        freq=train_df[col].value_counts()
        freq=pd.DataFrame(freq)
        freq.reset_index(inplace=True)
        freq.columns=[[col,col_freq]]

        # merge ths 'freq' datafarme with the train data
        temp_train_df=pd.merge(train_df[[col]], freq, how='left', on=col)
        temp_train_df.drop([col], axis=1, inplace=True)

        # merge this 'freq' dataframe with the test data
        temp_test_df=pd.merge(test_df[[col]], freq, how='left', on=col)
        temp_test_df.drop([col], axis=1, inplace=True)

        # if certain levels in the test dataset is not observed in the train dataset, 
        # we assign frequency of zero to them
        temp_test_df.fillna(0, inplace=True)
        temp_test_df[col_freq]=temp_test_df[col_freq].astype(np.int32)

        if result_train_df.shape[0]==0:
            result_train_df=temp_train_df
            result_test_df=temp_test_df
        else:
            result_train_df=pd.concat([result_train_df, temp_train_df],axis=1)
            result_test_df=pd.concat([result_test_df, temp_test_df],axis=1)
    
    return result_train_df, result_test_df


In [ ]:
cat_cols=['gender','city', 'is_auto_renew','is_cancel',
          'registered_via', 'is_discount', 'autorenew_&_not_cancel','notAutorenew_&_cancel']

In [ ]:
train.columns

In [ ]:
 #generate dataframe for frequency features for the train and test dataset
train_freq, test_freq=freq_encoding(cat_cols, train, test)


In [ ]:
train_freq.head()

In [ ]:
test_freq.head()

In [ ]:
# merge them into the original train and test dataset
train=pd.concat([train, train_freq], axis=1)
test=pd.concat([test,test_freq], axis=1)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
#train.to_csv('train.csv')
#test.to_csv('test.csv')

In [ ]:
train[train.columns[train.columns.str.contains('is_discount')]].head(5)

In [ ]:
train.shape,test.shape

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
del train['msno']
del test['msno']

In [ ]:
cat_cols=['gender','city', 'is_auto_renew','is_cancel',
          'registered_via', 'is_discount', 'autorenew_&_not_cancel','notAutorenew_&_cancel']

In [ ]:
tr = train.drop(cat_cols,axis=1)

In [ ]:
ts = test.drop(cat_cols,axis=1)

In [ ]:
tr.head()

In [ ]:
ts.head()

In [ ]:
del ts['is_churn']
is_churn_tr = tr['is_churn']
del tr['is_churn']

In [ ]:
tr.head()

In [ ]:
ts.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder ### currently not using ##
var_mod = [ 'registered_via_freq', 'gender_freq','payment_method_id', 'is_auto_renew_freq', 'is_cancel_freq',
          'is_discount_freq', 'autorenew_&_not_cancel_freq','notAutorenew_&_cancel_freq','city_freq','bd','registration_init_time',
          'amt_per_day','date','num_25', 'num_50','num_75', 'num_985','num_100', 'num_unq', 'total_secs']
le = LabelEncoder()
for i in var_mod:
    tr[i] = le.fit_transform(tr[i])

In [ ]:
for i in var_mod: ## Currently not using  ###
    ts[i] = le.fit_transform(ts[i])

In [ ]:
tr.dtypes

In [ ]:
ts.dtypes

In [ ]:
tr['is_churn'] = is_churn_tr ## tagged back to is churn label 

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
localtrain, localval=train_test_split(tr, test_size=0.25, random_state=2017)
y_localtrain=localtrain['is_churn']
x_localtrain=localtrain.drop(['is_churn'], axis=1)

y_localval=localval['is_churn']
x_localval=localval.drop('is_churn', axis=1)

In [ ]:
x_localtrain.shape,x_localval.shape,y_localtrain.shape,y_localval.shape

In [ ]:
import time 
from sklearn.ensemble import RandomForestClassifier

print('Start training...')
start_time=time.time()

rf=RandomForestClassifier(n_estimators=150, n_jobs=6, min_samples_split=5, max_depth=7,
                          criterion='gini', random_state=0)

rf.fit(x_localtrain, y_localtrain)
rf_valprediction=rf.predict_proba(x_localval)[:,1]

end_time = time.time()
print("it takes %.3f seconds to train and predict" % (end_time - start_time))

In [ ]:
from sklearn.metrics import roc_auc_score

rf_val_auc=roc_auc_score(y_localval, rf_valprediction)
rf_val_gininorm=2*rf_val_auc-1

print('Random Forest Validation AUC is {:.6f}'.format(rf_val_auc))
print('Random Forest Validation Normalised Gini Coefficient is {:.6f}'.format(rf_val_gininorm))

In [ ]:
x_test=ts

In [ ]:
scaler = StandardScaler().fit(x_localtrain.values)
x_localtrain_scaled = scaler.transform(x_localtrain)
x_localval_scaled = scaler.transform(x_localval)
x_test_scaled = scaler.transform(x_test)

print('Start training...')
start_time=time.time()

logit=LogisticRegression()
logit.fit(x_localtrain_scaled, y_localtrain)
logit_valprediction=logit.predict_proba(x_localval_scaled)[:,1]

end_time = time.time()
print("it takes %.3f seconds to train and predict" % (end_time - start_time))

In [ ]:
logit_val_auc=roc_auc_score(y_localval, logit_valprediction)
logit_val_gininorm=2*logit_val_auc-1

print('Logistic Regression Validation AUC is {:.6f}'.format(logit_val_auc))
print('Logistic Regression Validation Normalised Gini Coefficient is {:.6f}'.format(logit_val_gininorm))

In [ ]:
y_testprediction=logit.predict_proba(x_test_scaled)[:,1]


In [ ]:
y_testprediction

In [ ]:

#y_testprediction=rf.predict_proba(x_test)[:,1]


#rf_submission.to_csv('rf_submission.csv', compression='gzip', index=False)

In [ ]:
log_portu = pd.DataFrame({ 'msno': test_id,
                            'is_churn': y_testprediction })
log_portu.to_csv("logistic_portu_logic.csv", index=False)

plt.rcParams['figure.figsize'] = (10, 10)

# remove constant columns
colsToRemove = []
for col in train_s.columns:
    if train_s[col].std() == 0:
        colsToRemove.append(col)

train_s.head()

trainLabels = y
trainFeatures = train_s

In [ ]:
train_s.dtypes

test_id = test['msno']
del test['msno']
del train_s['msno']
del train_s['date']
del test['date']
del test['is_churn']

# Load in our libraries
import pandas as pd
import numpy as np
import re
import sklearn
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import warnings
warnings.filterwarnings('ignore')

# Going to use these 5 base models for the stacking
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.cross_validation import KFold;


colormap = plt.cm.RdBu
plt.figure(figsize=(25,25))
plt.title('Pearson Correlation of Features', y=1.05, size=15)
sns.heatmap(train_s.astype(float).corr(),linewidths=0.1,vmax=1.0, 
            square=True, cmap=colormap, linecolor='white', annot=True)

train_s.astype(float).corr()

train_2 = train_s[['autorenew_&_not_cancel', 'is_discount','plan_list_price','total_secs']]
test_2 =  test[['autorenew_&_not_cancel', 'is_discount','plan_list_price','total_secs']]

train_2.shape,test_2.shape

train_2.head()

# Put in our parameters for said classifiers
# Random Forest parameters
rf_params = {
    'n_jobs': -1,
    'n_estimators': 50,
     'warm_start': True, 
     #'max_features': 0.2,
    #'class_weights' = {0:0.2,1:0.8}
    'max_depth': 6,
    'min_samples_leaf': 2,
    'max_features' : 'sqrt',
    'verbose': 0
}

# Extra Trees Parameters
et_params = {
    'n_jobs': -1,
    'n_estimators':100,
    #'max_features': 0.5,
    'max_depth': 8,
    'min_samples_leaf': 2,
    'verbose': 0
}

# AdaBoost parameters
ada_params = {
    'n_estimators': 50,
    'learning_rate' : 0.75
}

# Gradient Boosting parameters
gb_params = {
    'n_estimators': 50,
     #'max_features': 0.2,
    'max_depth': 5,
    'min_samples_leaf': 2,
    'verbose': 0
}

# Create 5 objects that represent our 4 models
rf = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
et = SklearnHelper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)
ada = SklearnHelper(clf=AdaBoostClassifier, seed=SEED, params=ada_params)
gb = SklearnHelper(clf=GradientBoostingClassifier, seed=SEED, params=gb_params)

# Create Numpy arrays of train, test and target ( Survived) dataframes to feed into our models
y_train = train_s['is_churn'].ravel()
#train_s = train_s.drop(['is_churn'], axis=1)
x_train = train_2.values # Creates an array of the train data
x_test = test_2.values # Creats an array of the test data

x_train.shape,x_test.shape,y_train.shape

et_oof_train, et_oof_test = get_oof(et, x_train, y_train, x_test) # Extra Trees
rf_oof_train, rf_oof_test = get_oof(rf,x_train, y_train, x_test) # Random Forest
ada_oof_train, ada_oof_test = get_oof(ada, x_train, y_train, x_test) # AdaBoost 
gb_oof_train, gb_oof_test = get_oof(gb,x_train, y_train, x_test) # Gradient Boost

rf_feature = rf.feature_importances(x_train,y_train)
et_feature = et.feature_importances(x_train, y_train)
ada_feature = ada.feature_importances(x_train, y_train)

gb_feature = gb.feature_importances(x_train,y_train)

x_train = np.concatenate(( et_oof_train, rf_oof_train, ada_oof_train,gb_oof_train), axis=1)
x_test = np.concatenate(( et_oof_test, rf_oof_test, ada_oof_test,gb_oof_test), axis=1)

from sklearn.linear_model import LogisticRegression
model = LogisticRegression(class_weight={0:0.2, 1:0.7})
model = model.fit(x_train,y_train)

# check the accuracy on the training set
#model.score(X, y)

prediction = model.predict_proba(x_test)[:,1]
prediction

StackingSubmission3 = pd.DataFrame({ 'msno': test_id,
                            'is_churn': prediction })
StackingSubmission3.to_csv("StackingSubmission3.csv", index=False)

sns.distplot(StackingSubmission3.is_churn)

model = LogisticRegression(class_weight={0:0.1, 1:0.8})
model = model.fit(x_train,y_train)

prediction1 = model.predict_proba(x_test)[:,1]
prediction1

StackingSubmission4 = pd.DataFrame({ 'msno': test_id,
                            'is_churn': prediction1 })
StackingSubmission4.to_csv("StackingSubmission4.csv", index=False)

sns.distplot(StackingSubmission4.is_churn)